In [1]:
import shap
import sklearn
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


### Load the dataset

In [2]:
dfTrain = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
dfTest = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
dfMeta = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")
dfSubmit = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

### Explore the Dataset and deal with missing data

We will look here at the data, missing values and how to deal with them.

In [3]:
print("Train df shape: {}, Test df shape: {}".format(dfTrain.shape,dfTest.shape))
dfTrain.head()

Train df shape: (617, 58), Test df shape: (5, 57)


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


Some values in the output of the previous cell are marked as `NaN` ("not a number"). Let's examine the missing data pattern with `seaborn` heatmap.

In [4]:
# sns.heatmap(dfTrain.isnull(), cbar=False)
# plt.title("Training")
# plt.show()

In [5]:
columns_with_nan = dfTrain.columns[dfTrain.isna().any()].tolist()
nan_counts = dfTrain[columns_with_nan].isna().sum()
print("Number of NaNs for Train dataframe")
print(nan_counts)
columns_with_nan = dfTest.columns[dfTest.isna().any()].tolist()
nan_counts = dfTest[columns_with_nan].isna().sum()
print("Number of NaNs Test dataframe:")
print(nan_counts)

Number of NaNs for Train dataframe
BQ    60
CB     2
CC     3
DU     1
EL    60
FC     1
FL     1
FS     2
GL     1
dtype: int64
Number of NaNs Test dataframe:
Series([], dtype: float64)


No NaNs on the test dataframe, and train is not that bad. Let's use simple mean to replace NaNs in train df. And get our training, development and testing dataframes.

In [6]:
y_dev = dfTrain.Class
X_train = dfTrain.copy()
X_test = dfTest.copy()

#EJ is categorical, so we will transform it to 1 and ~0 (we will use rates and want to avoid division by 0) 
X_train = X_train.drop('Id', axis = 1)
X_train['EJ'] = X_train['EJ'].map({'A':0.001,'B':1})
X_train = X_train.apply(lambda col: col.fillna(col.mean()), axis=0)
X_test.drop('Id', axis = 1)
X_test['EJ'] = X_test['EJ'].map({'A':0.001,'B':1})

In [7]:
def groupCalculate(data):
    columns = data.columns
    index = 0
    new_columns = {}
    for column1 in columns:
        index += 1
        for column2 in columns[index:]:
            new_column_name = '{}/{}'.format(column1, column2)
            new_columns[new_column_name] = data[column1] / (data[column2] + 0.00001)
    
    new_columns_df = pd.DataFrame(new_columns)
    result = pd.concat([data, new_columns_df], axis=1)
    return result

In [8]:
corr_values = np.abs(X_train.corr().iloc[:, -1]).sort_values().index
X_dev = X_train[corr_values[:-1]]
X_test = X_test[corr_values[:-1]]

In [9]:
X_dev = groupCalculate(X_dev)
X_test = groupCalculate(X_test)

Split the dev set into a training and validation set,to train and tune our models, using a 75/25 split.

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.25, random_state=1)

In [11]:
#We will evaluate using concordance index
def concordance_index(event_observed, predicted_scores):
    if isinstance(event_observed, (pd.DataFrame, pd.Series)):
        event_observed = event_observed.values.flatten()
    if isinstance(predicted_scores, (pd.DataFrame, pd.Series)):
        predicted_scores = predicted_scores.values.flatten()
    
    n = len(event_observed)
    assert n == len(predicted_scores)
    
    num_pairs = 0
    num_correct_pairs = 0
    
    for i in range(n):
        for j in range(i + 1, n):
            if event_observed[i] != event_observed[j]:
                num_pairs += 1
                if (event_observed[i] > event_observed[j] and predicted_scores[i] > predicted_scores[j]) or \
                   (event_observed[i] < event_observed[j] and predicted_scores[i] < predicted_scores[j]):
                    num_correct_pairs += 1
                elif predicted_scores[i] == predicted_scores[j]:
                    num_correct_pairs += 0.5
    
    return num_correct_pairs / num_pairs if num_pairs > 0 else np.nan

In [12]:
#Just to test initially we use a decision tree classifier
# dt_hyperparams = {
#     'max_depth': 7,
#     'max_leaf_nodes':40
# }
# dt = DecisionTreeClassifier(**dt_hyperparams, random_state=10)
# dt.fit(X_train, y_train)

# y_train_preds = dt.predict_proba(X_train)[:, 1]
# print(f"Train C-Index: {concordance_index(y_train.values, y_train_preds)}")
# y_val_preds = dt.predict_proba(X_val)[:, 1]
# print(f"Val C-Index: {concordance_index(y_val.values, y_val_preds)}")
# print(f"mio: {concordance_index(y_train.values, y_train_preds)+concordance_index(y_val.values, y_val_preds)}")

In [13]:
#Random forest are the go to solution for this kind of prognostics
# rf = RandomForestClassifier(n_estimators=100, random_state=10)
# rf.fit(X_train, y_train)

# y_train_rf_preds = rf.predict_proba(X_train)[:, 1]
# print(f"Train C-Index: {concordance_index(y_train.values, y_train_rf_preds)}")

# y_val_rf_preds = rf.predict_proba(X_val)[:, 1]
# print(f"Val C-Index: {concordance_index(y_val.values, y_val_rf_preds)}")
# print(f"mio: {concordance_index(y_train.values, y_train_rf_preds)+concordance_index(y_val.values, y_val_rf_preds)}")

I took this functions from Coursera's trainig, are quite goot to estimate hyperparameters, finding a model that both has good predictive performance and minimizes overfitting.

The hyperparameters to adjust are:

- `n_estimators`: the number of trees used in the forest.
- `max_depth`: the maximum depth of each tree.
- `min_samples_leaf`: the minimum number (if `int`) or proportion (if `float`) of samples in a leaf.

The approach we implement to tune the hyperparameters is known as a grid search:

- We define a set of possible values for each of the target hyperparameters.

- A model is trained and evaluated for every possible combination of hyperparameters.

- The best performing set of hyperparameters is returned.

The cell below implements a hyperparameter grid search, using the C-Index to evaluate each tested model.

In [14]:
def holdout_grid_search(clf, X_train_hp, y_train_hp, X_val_hp, y_val_hp, hyperparams, fixed_hyperparams={}):
    '''
    Conduct hyperparameter grid search on hold out validation set. Use holdout validation.
    Hyperparameters are input as a dictionary mapping each hyperparameter name to the
    range of values they should iterate over. Use the cindex function as your evaluation
    function.

    Input:
        clf: sklearn classifier
        X_train_hp (dataframe): dataframe for training set input variables
        y_train_hp (dataframe): dataframe for training set targets
        X_val_hp (dataframe): dataframe for validation set input variables
        y_val_hp (dataframe): dataframe for validation set targets
        hyperparams (dict): hyperparameter dictionary mapping hyperparameter
                            names to range of values for grid search
        fixed_hyperparams (dict): dictionary of fixed hyperparameters that
                                  are not included in the grid search
    Output:
        best_estimator (sklearn classifier): fitted sklearn classifier with best performance on
                                             validation set
        best_hyperparams (dict): hyperparameter dictionary mapping hyperparameter
                                 names to values in best_estimator
    '''
    best_estimator = None
    best_hyperparams = {}
    best_score = 0.0
    lists = hyperparams.values()
    param_combinations = list(itertools.product(*lists))
    total_param_combinations = len(param_combinations)
    for i, params in enumerate(param_combinations, 1):
        param_dict = {}
        for param_index, param_name in enumerate(hyperparams):
            param_dict[param_name] = params[param_index]
        estimator = clf(**param_dict, **fixed_hyperparams)
        estimator.fit(X_train_hp, y_train_hp)
        preds = estimator.predict_proba(X_val_hp)
        estimator_score = concordance_index(y_val_hp, preds[:,1])

        print(f'[{i}/{total_param_combinations}] {param_dict}')
        print(f'Val C-Index: {estimator_score}\n')
        if estimator_score >= best_score:
                best_score = estimator_score
                best_estimator = estimator
                best_hyperparams = param_dict
    best_hyperparams.update(fixed_hyperparams)
    return best_estimator, best_hyperparams

In [15]:
#I have tested in a larger grid, leaving here only a couple of good values.
def random_forest_grid_search(X_train, y_train, X_val, y_val):
    hyperparams = {
        'n_estimators': [990, 1000],
        'max_depth': [100000, 200000],
        'min_samples_leaf': [1, 2],
    }
    fixed_hyperparams = {
        'random_state': 10,
    }
    rf = RandomForestClassifier
    best_rf, best_hyperparams = holdout_grid_search(rf, X_train, y_train,
                                                    X_val, y_val, hyperparams,
                                                    fixed_hyperparams)
    print(f"Best hyperparameters:\n{best_hyperparams}")
    y_train_best = best_rf.predict_proba(X_train)[:, 1]
    print(f"Train C-Index: {concordance_index(y_train, y_train_best)}")
    y_val_best = best_rf.predict_proba(X_val)[:, 1]
    print(f"Val C-Index: {concordance_index(y_val, y_val_best)}")
    best_hyperparams.update(fixed_hyperparams)
    
    return best_rf, best_hyperparams

In [16]:
# best_rf, best_hyperparams = random_forest_grid_search(X_train, y_train, X_val, y_val)

In [17]:
best_hyperparams = {
    'n_estimators': 1000,
    'max_depth': 200000,
    'min_samples_leaf': 2,
    'random_state': 10,
}
model = RandomForestClassifier(**best_hyperparams)
model.fit(X_dev,y_dev)
preds = model.predict_proba(X_test)

In [18]:
dfSubmit[['class_0', 'class_1']] = np.mean(preds, axis=0)
dfSubmit.to_csv('/kaggle/working/submission.csv', index=False)